In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from itertools import permutations
import pandas as pd
import numpy as np
import tqdm

In [11]:
# Load TRAIN / TEST data from drive
data = pd.read_csv('/content/drive/MyDrive/Machine Learning/Zero Shot Learning/NTECH/CIFAR/Dataset/CIFAR_100_TRAIN_FEAT100VGG16')

In [ ]:
normalized_df=(data[data.columns[:-1]]-data[data.columns[:-1]].min())/(data[data.columns[:-1]].max()-data[data.columns[:-1]].min())

In [ ]:
normalized_df['class'] = data['class']
data = normalized_df.copy()

In [ ]:
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,class
0,0.383867,0.454235,0.353458,0.585264,0.397395,0.736901,0.309836,0.505442,0.607276,0.632240,0.634145,0.426886,0.610184,0.536241,0.647633,0.698377,0.675315,0.343024,0.520127,0.794277,0.414295,0.603700,0.240358,0.365602,0.432570,0.364740,0.241566,0.274504,0.393730,0.713391,0.696341,0.678160,0.525920,0.538111,0.515298,0.744610,0.167702,0.435843,0.649837,0.739416,...,0.357040,0.282207,0.633860,0.212740,0.318323,0.743588,0.238258,0.653784,0.930624,0.888129,0.073810,0.549944,0.255060,0.324644,0.475199,0.251482,0.739479,0.523148,0.374165,0.718747,0.426464,0.304625,0.808693,0.460756,0.337769,0.520379,0.828995,0.196316,0.200268,0.456236,0.337902,0.617170,0.367963,0.161300,0.665409,0.524193,0.472691,0.642029,0.684476,19
1,0.358692,0.484090,0.347046,0.605324,0.373786,0.649619,0.291694,0.440285,0.598675,0.601138,0.676270,0.522285,0.606522,0.639947,0.616811,0.684916,0.715627,0.353499,0.608422,0.852163,0.432791,0.626796,0.203549,0.363692,0.452615,0.338151,0.301861,0.325108,0.363589,0.769743,0.688774,0.697211,0.550127,0.531306,0.499183,0.727682,0.194902,0.483926,0.611647,0.678394,...,0.336735,0.254618,0.575829,0.156154,0.323661,0.752368,0.291216,0.711387,0.901559,0.898788,0.100422,0.462591,0.237266,0.335381,0.509811,0.306731,0.737258,0.567506,0.403900,0.651424,0.378594,0.333148,0.779113,0.469570,0.349345,0.525941,0.858293,0.276538,0.082721,0.468616,0.400731,0.563535,0.456319,0.237894,0.706146,0.440498,0.465596,0.691910,0.722493,29
2,0.381934,0.494994,0.358185,0.629300,0.335250,0.666680,0.272157,0.444930,0.587516,0.611633,0.667048,0.535455,0.619773,0.622445,0.615522,0.689258,0.712860,0.335771,0.645539,0.847690,0.439452,0.611150,0.193330,0.369981,0.479032,0.348037,0.302363,0.340559,0.386891,0.747122,0.684064,0.674727,0.560804,0.507195,0.501648,0.738074,0.206238,0.492749,0.598863,0.658550,...,0.342200,0.270425,0.573341,0.167883,0.341141,0.739249,0.304918,0.708945,0.909446,0.883282,0.106389,0.459459,0.234414,0.332053,0.530146,0.286126,0.724057,0.578246,0.396326,0.653228,0.388742,0.324137,0.765990,0.462731,0.372748,0.527017,0.859336,0.282394,0.089373,0.468760,0.395112,0.554047,0.460380,0.243188,0.712175,0.454467,0.449259,0.662808,0.725370,0
3,0.440725,0.518507,0.440582,0.566451,0.432550,0.717395,0.310423,0.436956,0.591646,0.593044,0.770377,0.558007,0.635375,0.376656,0.607131,0.690683,0.720614,0.470562,0.588311,0.752360,0.432635,0.458711,0.307979,0.309693,0.535998,0.305252,0.416001,0.443648,0.424397,0.565783,0.614196,0.592334,0.519975,0.361795,0.472587,0.596520,0.379222,0.341604,0.561583,0.503547,...,0.316511,0.379142,0.711501,0.208848,0.385683,0.638348,0.359778,0.527557,0.778246,0.692661,0.284891,0.348490,0.207931,0.347548,0.448547,0.311242,0.614927,0.601186,0.331123,0.715491,0.388262,0.451286,0.740190,0.481965,0.336193,0.539002,0.748300,0.330040,0.131687,0.483841,0.432479,0.468527,0.401859,0.379764,0.736405,0.504841,0.378757,0.485221,0.689327,11
4,0.582187,0.522613,0.317755,0.468635,0.720297,0.695304,0.523526,0.523191,0.651658,0.705031,0.633902,0.368527,0.498152,0.424839,0.658309,0.619861,0.598573,0.470774,0.427365,0.703060,0.551636,0.407752,0.391170,0.286392,0.472490,0.365630,0.509500,0.425849,0.659725,0.601933,0.572157,0.604139,0.353335,0.542257,0.573943,0.601277,0.440924,0.510913,0.457701,0.485620,...,0.488197,0.552612,0.578213,0.243944,0.329973,0.650735,0.463622,0.539823,0.686134,0.505834,0.365500,0.655181,0.431356,0.316161,0.451839,0.234407,0.478804,0.337402,0.431523,0.721647,0.400492,0.387855,0.578559,0.582340,0.363547,0.374498,0.559359,0.279534,0.301453,0.614196,0.289203,0.520182,0.288367,0.368138,0.583381,0.573061,0.405141,0.565814,0.635721,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [12]:
# Permutation to find unique combination of classes
x = list(range(100))
perm = list(permutations(x, 2))

In [13]:
# Map index to class names
'''
Content of the CIFAR 100 Dataset
'''
idx2label={
0: 'apple',
1: 'fish',
2: 'baby',
3: 'bear',
4: 'beaver',
5: 'bed',
6: 'bee',
7: 'beetle',
8: 'bicycle',
9: 'bottle',
10: 'bowl',
11: 'boy',
12: 'bridge',
13: 'bus',
14: 'butterfly',
15: 'camel',
16: 'can',
17: 'castle',
18: 'caterpillar',
19: 'cattle',
20: 'chair',
21: 'chimpanzee',
22: 'clock',
23: 'cloud',
24: 'cockroach',
25: 'couch',
26: 'crab',
27: 'crocodile',
28: 'cup',
29: 'dinosaur',
30: 'dolphin',
31: 'elephant',
32: 'flatfish',
33: 'forest',
34: 'fox',
35: 'girl',
36: 'hamster',
37: 'house',
38: 'kangaroo',
39: 'keyboard',
40: 'lamp',
41: 'mower',
42: 'leopard',
43: 'lion',
44: 'lizard',
45: 'lobster',
46: 'man',
47: 'maple',
48: 'motorcycle',
49: 'mountain',
50: 'mouse',
51: 'mushroom',
52: 'oak',
53: 'orange',
54: 'orchid',
55: 'otter',
56: 'palm',
57: 'pear',
58: 'truck',
59: 'pine',
60: 'plain',
61: 'plate',
62: 'poppy',
63: 'porcupine',
64: 'possum',
65: 'rabbit',
66: 'raccoon',
67: 'ray',
68: 'road',
69: 'rocket',
70: 'rose',
71: 'sea',
72: 'seal',
73: 'shark',
74: 'shrew',
75: 'skunk',
76: 'skyscraper',
77: 'snail',
78: 'snake',
79: 'spider',
80: 'squirrel',
81: 'streetcar',
82: 'sunflower',
83: 'pepper',
84: 'table',
85: 'tank',
86: 'telephone',
87: 'television',
88: 'tiger',
89: 'tractor',
90: 'train',
91: 'trout',
92: 'tulip',
93: 'turtle',
94: 'wardrobe',
95: 'whale',
96: 'tree',
97: 'wolf',
98: 'woman',
99: 'worm'
}

In [6]:
# Download Word2Vec 300 dim
import gensim.downloader as api
from scipy.spatial import distance
wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [7]:
wordvec ={}
def retVec(idx):
  #a= wv.get_vector(idx2label[idx])
  wordvec[idx] = wv.get_vector(idx2label[idx]) #np.array([(float(i)-min(a))/(max(a)-min(a)) for i in a])
for i in range(100):
  retVec(i)


In [18]:
(1-distance.cosine(wv.get_vector('apple'),wv.get_vector('sea'))),(1-distance.cosine(wv.get_vector('apple'),wv.get_vector('pear')))

(0.011174865067005157, 0.6450695991516113)

In [14]:
# With added features
def cosine_sim_feat(v1,v2):
  return 1-distance.cosine(v1, v2)

# Add mean featuers
def makeVec(idx):
  
  vec =np.array(list(data[data['class']==idx].mean(axis=0))[:-1]) # Add mean (100 dim) of the referenced class (index)
  return vec

cls1 = []
cls2 = []
cosine_sim = []

for combos in perm:
  cls1.append(idx2label[combos[0]])
  cls2.append(idx2label[combos[1]])
  cosine_sim.append( cosine_sim_feat( makeVec(combos[0]),makeVec(combos[1]) ) )

In [15]:
df_dict ={
    'Class 1':cls1,
    'Class 2':cls2,
    'Cosine Sim': cosine_sim
}
pd.DataFrame.from_dict(df_dict).to_csv('CIFAR100_TRAINFEAT_SIM.csv',index=False)

In [17]:
# Returns the corresponding word vector


# Returs cosine similarities between two word vectors
def cosine_sim(idx1,idx2):
  return 1-distance.cosine(wordvec[idx1],wordvec[idx2])

# With added features
def cosine_sim_feat(wv1,wv2):
  return 1-distance.cosine(wv1, wv2)

# Add mean featuers
def makeVec(idx):
  vec = wordvec[idx]  # Get 300 dim word vector
  vec =np.append(vec, np.array(list(data[data['class']==idx].mean(axis=0))[:-1])) # Add mean (100 dim) of the referenced class (index)
  print(vec.size)
  return vec



cls1 = []
cls2 = []
cosine_sim_without_feat = []
cosine_sim_wit_feat = []

for combos in perm:
  cls1.append(idx2label[combos[0]])
  cls2.append(idx2label[combos[1]])
  cosine_sim_without_feat.append( cosine_sim( combos[0],combos[1] ) )
  cosine_sim_wit_feat.append( cosine_sim_feat( makeVec(combos[0]),makeVec(combos[1]) ) )
  break

400
400


In [9]:
df_dict ={
    'Class 1':cls1,
    'Class 2':cls2,
    'Sematic Cosine Similarity Without VGG Feat': cosine_sim_without_feat,
    'Sematic Cosine Similarity Wit VGG Feat': cosine_sim_wit_feat
}

In [10]:
pd.DataFrame.from_dict(df_dict).to_csv('Cosine_Sim_CIFAR100_TRAIN_v4.csv',index=False)